# Loading in csv file with photometry results

In [77]:
#import statements
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import ascii
from astropy.table import join
from astropy.table import Table, vstack

#read in csv file with photometry results
phot=ascii.read('/users/adignan/csv/photometry_results.csv',format='csv')

#modify both Tables (rename columns, calculate nondetections vs detections)
phot['SNR > 3?'] = ['YES' if x > 3 else 'NO' for x in phot['SNR']]
photmod=phot[['Source ID','freq (GHz)','flux density (mJy)','error (mJy)','SNR > 3?']]

names=('Source ID','freq (GHz)','flux density (mJy)','error (mJy)','SNR > 3?')
new_names=('sourceid','freq','flux','error','snr_ratio')

photmod.rename_columns(names,new_names)

#make a dataframe version of our astropy Table so we can do conditional stuff below
photmod=Table.to_pandas(photmod)

# np.select and define list of condition with corresponding values
photmod['data']=(np.select([photmod['snr_ratio'].eq('YES'), # condition #1
                         photmod['snr_ratio'].eq('NO')],# condition #2
                        [photmod['flux'],            # value when #1 is true
                         3*photmod['error']]))           # value when #2 is true

photmod['data_err']=(np.select([photmod['snr_ratio'].eq('YES'), # condition #1
                         photmod['snr_ratio'].eq('NO')], # condition #2
                        [photmod['error'], # value when #1 is true
                         0.0]))        # value when #2 is true

photmod['lims']=(np.select([photmod['snr_ratio'].eq('YES'), # condition #1
                        photmod['snr_ratio'].eq('NO')], # condition #2
                        [False, # value when #1 is true
                         True]))       # value when #2 is true

photmod['marker']=(np.select([photmod['snr_ratio'].eq('YES'), # condition #1
                         photmod['snr_ratio'].eq('NO')], # condition #2
                        ['o', # value when #1 is true
                         'o']))       # value when #2 is true

#############################################################################################

#read in csv file with photometry results
# old_df = pd.read_csv('/users/adignan/csv/NEW_photometry_results.csv')
# # old_data=old_df[old_df['source ID'].str.contains('NGC2146')] #we want just NGC5194 sources!

# oldphot=Table.from_pandas(old_data)

# #modify both Tables (rename columns, calculate nondetections vs detections)
# oldphot['SNR > 3?'] = ['YES' if x > 3 else 'NO' for x in oldphot['SNR']]
# oldphotmod=oldphot[['source ID','freq (GHz)',
#                     'flux density (mJy)','error (mJy)','SNR > 3?']]

# old_new_names=('sourceid','freq','old_flux','old_error','old_snr_ratio')

# oldphotmod.rename_columns(names,old_new_names)

# #make a dataframe version of our astropy Table so we can do conditional stuff below
# oldphotmod=Table.to_pandas(oldphotmod)

# # np.select and define list of condition with corresponding values
# oldphotmod['old_data']=(np.select([oldphotmod['old_snr_ratio'].eq('YES'), # condition #1
#                          oldphotmod['old_snr_ratio'].eq('NO')],# condition #2
#                         [oldphotmod['old_flux'],            # value when #1 is true
#                          3*oldphotmod['old_error']]))           # value when #2 is true

# oldphotmod['old_data_err']=(np.select([oldphotmod['old_snr_ratio'].eq('YES'), # condition #1
#                          oldphotmod['old_snr_ratio'].eq('NO')], # condition #2
#                         [oldphotmod['old_error'], # value when #1 is true
#                          0.0]))        # value when #2 is true

# oldphotmod['old_lims']=(np.select([oldphotmod['old_snr_ratio'].eq('YES'), # condition #1
#                         oldphotmod['old_snr_ratio'].eq('NO')], # condition #2
#                         [False, # value when #1 is true
#                          True]))       # value when #2 is true

# oldphotmod['old_marker']=(np.select([oldphotmod['old_snr_ratio'].eq('YES'), # condition #1
#                          oldphotmod['old_snr_ratio'].eq('NO')], # condition #2
#                         ['s', # value when #1 is true
#                          's']))       # value when #2 is true

# # ids=['NGC0337a','NGC0337b','NGC0337d]

# # df_filt = df[df.sourceid.isin(ids)]

# merged_df = photmod.merge(oldphotmod, how = 'inner', 
#                           on = ['sourceid','freq'])

# merged_df
#go ahead and remove NGC4631Enuc.1 from our table so it doesn't cause trouble later on
# photmod=photmod[photmod['sourceid']!='NGC4631Enuc.1']
photmod.to_csv('/users/adignan/photmod_july.csv')

# Creating plots of fluxes for each source

## Spectral line index fitting

In [86]:
from scipy.optimize import curve_fit
import sys
from itertools import repeat

alpha1vals=[]
b1vals=[]
alpha1err=[]

alpha2vals=[]
b2vals=[]
alpha2err=[]

alphavals=[]
bvals=[]

def func_fit(a,x,b):
    return a*x + b

for i in range(photmod.sourceid.nunique()):
        
        #3 to 33 GHz
        data1=photmod[i*4:i*4+3]
        index1 = data1[data1['snr_ratio'] == 'NO'].index

        #33 to 90 GHz
        data2=photmod[i*4+2:i*4+4]
        index2 = data2[data2['snr_ratio'] == 'NO'].index
        
        #3 to 90 GHz
        data3=photmod[i*4:i*4+4]
        index3 = data3[data3['snr_ratio'] == 'NO'].index
        
        #don't fit for 3 to 33 GHz if we don't have enough detections
        if ((len(index1)) == 3) or ((len(index1)) == 2):
            popt1 = [np.nan, np.nan]
            pcov1 = np.empty((2,2))
            pcov1[:] = np.nan
        #fit if we have enough detections
        if ((len(index1)) == 1) or ((len(index1)) == 0):
            data1=data1.drop(index1,axis=0)
            popt1, pcov1 = curve_fit(f=func_fit, xdata=np.log10(data1.freq.tolist()), ydata=np.log10(data1.flux.tolist()),sigma=0.434*data1.error/data1.flux) 
            
        #don't fit for 33 to 90 GHz if we don't have enough detections        
        if (len(index2)) != 0:
            popt2 = [np.nan, np.nan]
            pcov2 = np.empty((2,2))
            pcov2[:] = np.nan
        #fit if we have enough detections
        else:
            popt2, pcov2 = curve_fit(f=func_fit, xdata=np.log10(data2.freq.tolist()), ydata=np.log10(data2.flux.tolist()),sigma=0.434*data2.error/data2.flux)
    
        #don't fit for 3 to 90 GHz if we don't have enough detections
        if ((len(index3)) == 4) or ((len(index3)) == 3):
            popt3 = [np.nan, np.nan]
            pcov3 = np.empty((2,2))
            pcov3[:] = np.nan
        #fit if we have enough detections
        if ((len(index3)) == 2) or ((len(index3)) == 1) or ((len(index3)) == 0):
            data3=data3.drop(index3,axis=0)
            popt3, pcov3 = curve_fit(f=func_fit, xdata=np.log10(data3.freq.tolist()), ydata=np.log10(data3.flux.tolist()),sigma=0.434*data3.error/data3.flux)

        #unpack the fitting results
        alpha1,b1=popt1
        alpha2,b2=popt2
        alpha3,b3=popt3
        
        #save unpacked results!
        alpha1vals.append(alpha1)
        b1vals.append(b1)
            
        alpha2vals.append(alpha2)
        b2vals.append(b2)
        
        alphavals.append(alpha3)
        bvals.append(b3)

        i=+4
        
#make a dataframe of the results
print(len(photmod['sourceid'].unique()))
specdf = pd.DataFrame(np.column_stack([photmod['sourceid'].unique(), 
                                       alpha1vals, b1vals, alpha2vals, b2vals, 
                                       alphavals, bvals]), 
                               columns=['source ID', 'alpha1', 'b1',
                                        'alpha2','b2', 'alpha', 'b'])

#save dataframe as csv
specdf.to_csv('/users/adignan/specdf_july.csv')
print(type(specdf))
# photmod

/users/adignan/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


120
<class 'pandas.core.frame.DataFrame'>


## Actual plotting

In [96]:
def func_plot(x,a,b):
    return (x**a) * (10**b)

grouped_df = photmod.groupby('sourceid')

for key, item in grouped_df:
    print(key,item, "\n\n")

NGC0337a    sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
0  NGC0337a     3  1.720551  0.134939       YES  1.720551  0.134939     0   
1  NGC0337a    15  1.252183  0.124283       YES  1.252183  0.124283     0   
2  NGC0337a    33  0.604797  0.109020       YES  0.604797  0.109020     0   
3  NGC0337a    90  0.599209  0.107580       YES  0.599209  0.107580     0   

  marker  
0      o  
1      o  
2      o  
3      o   


NGC0337b    sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
4  NGC0337b     3  4.663468  0.134980       YES  4.663468  0.134980     0   
5  NGC0337b    15  2.532913  0.125145       YES  2.532913  0.125145     0   
6  NGC0337b    33  1.475663  0.114852       YES  1.475663  0.114852     0   
7  NGC0337b    90  1.528148  0.106123       YES  1.528148  0.106123     0   

  marker  
4      o  
5      o  
6      o  
7      o   


NGC0337c     sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
8   NGC

In [85]:
def func_plot(x,a,b):
    return (x**a) * (10**b)

for i in range(photmod.sourceid.nunique()):
    
        data=photmod[i*4:i*4+4]
        print(data)
        
#         print(data)
        
#         title=merged_df.iloc[i*4:i*4+4]
    
        plt.figure()

        #plot new data
        for idx in data.index.tolist():
            plt.errorbar(data.freq[idx],data.data[idx],yerr=data.error[idx],marker=data.marker[idx],uplims=data.lims[idx])
            
        if np.isnan(specdf.alpha1[i]):
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha1[i],x=np.array(data.freq),b=specdf.b1[i]),
                    label='_nolegend_')
        else:
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha1[i],x=np.array(data.freq),b=specdf.b1[i]),
                    label='$α_{3–33 GHz}$'+'='+'{:0.2}'.format(specdf.alpha1[i])+'±'+'{:0.2}'.format(specdf.b1[i]))

        if np.isnan(specdf.alpha2[i]):
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha2[i],x=np.array(data.freq),b=specdf.b2[i]),
                    label='_nolegend_')
        else:
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha2[i],x=np.array(data.freq),b=specdf.b2[i]),
                    label='$α_{33–90 GHz}$'+'='+'{:0.2}'.format(specdf.alpha2[i])+'±'+'{:0.2}'.format(specdf.b2[i]))

        #plot old data
#         for idx in data.index.tolist():
#             plt.errorbar(data.freq[idx],data.old_data[idx],yerr=data.old_error[idx],marker=data.old_marker[idx],uplims=data.old_lims[idx])

        plt.xticks([3,15,33,90])
        plt.xlabel('frequency (GHz)')
        plt.ylabel('flux density (mJy)')
        plt.yscale('log')
        plt.xscale('log')
        plt.title(data.sourceid[i*4])
        plt.legend()

        plt.savefig('/users/adignan/specfits/'+str(photmod.sourceid[i*4])+'.png',bbox_inches = "tight")

        plt.close('all')

        i=+4

   sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
0  NGC0337a     3  1.720551  0.134939       YES  1.720551  0.134939     0   
1  NGC0337a    15  1.252183  0.124283       YES  1.252183  0.124283     0   
2  NGC0337a    33  0.604797  0.109020       YES  0.604797  0.109020     0   
3  NGC0337a    90  0.599209  0.107580       YES  0.599209  0.107580     0   

  marker  
0      o  
1      o  
2      o  
3      o  
   sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
4  NGC0337b     3  4.663468  0.134980       YES  4.663468  0.134980     0   
5  NGC0337b    15  2.532913  0.125145       YES  2.532913  0.125145     0   
6  NGC0337b    33  1.475663  0.114852       YES  1.475663  0.114852     0   
7  NGC0337b    90  1.528148  0.106123       YES  1.528148  0.106123     0   

  marker  
4      o  
5      o  
6      o  
7      o  
    sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
8   NGC0337c     3  0.723346  0.135221  

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


   sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
16  NGC0628     3 -0.017864  0.087459        NO  0.262376       0.0     1   
17  NGC0628    15  0.065607  0.152527        NO  0.457580       0.0     1   
18  NGC0628    33  0.052887  0.086878        NO  0.260635       0.0     1   
19  NGC0628    90 -0.016211  0.075912        NO  0.227737       0.0     1   

   marker  
16      o  
17      o  
18      o  
19      o  
         sourceid  freq      flux     error snr_ratio      data  data_err  \
20  NGC0628Enuc.1     3  1.166958  0.088897       YES  1.166958  0.088897   
21  NGC0628Enuc.1    15  0.972580  0.151017       YES  0.972580  0.151017   
22  NGC0628Enuc.1    33  0.480006  0.110756       YES  0.480006  0.110756   
23  NGC0628Enuc.1    90  0.230996  0.089480        NO  0.268439  0.000000   

    lims marker  
20     0      o  
21     0      o  
22     0      o  
23     1      o  
         sourceid  freq      flux     error snr_ratio      data  data_err  \
2

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


   sourceid  freq      flux     error snr_ratio      data  data_err  lims  \
72  NGC3521     3 -0.107496  0.125046        NO  0.375137  0.000000     1   
73  NGC3521    15 -0.102080  0.229382        NO  0.688147  0.000000     1   
74  NGC3521    33 -0.101250  0.099563        NO  0.298690  0.000000     1   
75  NGC3521    90  0.409031  0.124748       YES  0.409031  0.124748     0   

   marker  
72      o  
73      o  
74      o  
75      o  


...

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


          sourceid  freq  flux  error snr_ratio  data  data_err  lims marker
428  NGC4631Enuc.1     3   NaN    NaN        NO   NaN       0.0     1      o
429  NGC4631Enuc.1    15   NaN    NaN        NO   NaN       0.0     1      o
430  NGC4631Enuc.1    33   NaN    NaN        NO   NaN       0.0     1      o
431  NGC4631Enuc.1    90   NaN    NaN        NO   NaN       0.0     1      o


ValueError: Data has no positive values, and therefore cannot be log-scaled.

ValueError: Data has no positive values, and therefore cannot be log-scaled.

<Figure size 640x480 with 1 Axes>

In [ ]:
def func_plot(x,a,b):
    return (x**a) * (10**b)

for i in range(photmod.sourceid.nunique()):

        data=photmod[i*4:i*4+4]
        title=photmod.iloc[i*4:i*4+4]
    
        plt.figure()


        for idx in data.index.tolist():
            plt.errorbar(data.freq[idx],data.data[idx],yerr=data.error[idx],marker=data.marker[idx],uplims=data.lims[idx])

        if np.isnan(specdf.alpha1[i]):
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha1[i],x=np.array(data.freq),b=specdf.b1[i]),
                    label='_nolegend_')
        else:
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha1[i],x=np.array(data.freq),b=specdf.b1[i]),
                    label='$α_{3–33 GHz}$'+'='+'{:0.2}'.format(specdf.alpha1[i])+'±'+'{:0.2}'.format(specdf.b1[i]))

        if np.isnan(specdf.alpha2[i]):
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha2[i],x=np.array(data.freq),b=specdf.b2[i]),
                    label='_nolegend_')
        else:
            plt.plot(np.array(data.freq),func_plot(a=specdf.alpha2[i],x=np.array(data.freq),b=specdf.b2[i]),
                    label='$α_{33–90 GHz}$'+'='+'{:0.2}'.format(specdf.alpha2[i])+'±'+'{:0.2}'.format(specdf.b2[i]))

    #         if np.isnan(specdf.alpha[i]):
    #             plt.plot(np.array(data.freq),func_plot(a=specdf.alpha[i],x=np.array(data.freq),b=specdf.b[i]),
    #                     label='_nolegend_')
    #         else:
    #             plt.plot(np.array(data.freq),func_plot(a=specdf.alpha[i],x=np.array(data.freq),b=specdf.b[i]),
    #                     label='$α_{3–90 GHz}$'+'='+'{:0.2}'.format(specdf.alpha[i])+'±'+'{:0.2}'.format(specdf.b[i]))


        plt.xticks([3,15,33,90])
        plt.xlabel('frequency (GHz)')
        plt.ylabel('flux density (mJy)')
        plt.yscale('log')
        plt.xscale('log')
        plt.title(title['sourceid'])
        plt.legend()

        plt.savefig('/users/adignan/'+str(title['sourceid'])+'_combined.png',bbox_inches = "tight")

        plt.close('all')

        i=+4

# Reproducing Abigail's histogram of spectral index values

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

specdf=pd.read_csv('/users/adignan/csv/specdfcomplete.csv')

bins=np.histogram(np.hstack((specdf['alpha1'].dropna(),specdf['alpha2'].dropna())), bins=10)[1] 


plt.hist(specdf['alpha1'].dropna(),label='$α_{3–33 GHz}$',histtype='step',color='#1f77b4',bins=bins)
plt.hist(specdf['alpha2'].dropna(),label='$α_{33–90 GHz}$',histtype='step',color='#ff7f0e',bins=bins)

plt.legend()
# plt.title('bins=10 (default)')
plt.xlabel('spectral index (α)')
plt.ylabel('number')
plt.axvline(np.median(specdf['alpha1'].dropna()),label='median $α_{3–33 GHz}$: -0.258',
            color='#1f77b4',linestyle='-.')
plt.axvline(np.median(specdf['alpha2'].dropna()),label='median $α_{33–90 GHz}$: -0.113',
            color='#ff7f0e',linestyle='-.')
# plt.axvline(np.median(specdf['alpha']),label='median $α_{3–90 GHz}$')
plt.legend()

print(np.median(specdf['alpha1']))
print(np.median(specdf['alpha2'].dropna()))
# print(np.median(specdf['alpha']))

#'#1f77b4', '#ff7f0e', '#2ca02c',

print(len(specdf))

# Calculating thermal fraction based on fitting results

In [50]:
from astropy.io import ascii
from astropy.table import join
from astropy.table import Table, vstack, Column
from astropy.stats import median_absolute_deviation
import numpy as np
import math

x=float('nan')

# table1=ascii.read('/users/adignan/specdf_take2.csv',format='csv')
# table2=ascii.read('/users/adignan/newspecdf.csv',format='csv')
# table3=vstack([table1, table2], join_type='exact')
# table3.write('/users/adignan/specdfcomplete.csv',overwrite=True)
table=ascii.read('/users/adignan/specdf_july.csv',format='csv')

def tfrac(alpha):
    fraction=( (3/33)**(-alpha) - (3/33)**(0.85) )/( (3/33)**(-0.1) - (3/33)**(0.85) )
    return fraction 

def tfrac_unc(alpha,unc):
    # Constants
    nu1 = 33  # GHz
    nu2 = 3   # GHz
    alpha_NT = -0.85 
    sigma_alpha_NT = 0
    # Calculated values
    A = (nu2 / nu1) ** (-alpha)
    B = (nu2 / nu1) ** (-alpha_NT)
    C = (nu2 / nu1) ** (-0.1)
    ln_term = np.log(nu2 / nu1)

    # Partial derivatives
    df_dalpha = (A * ln_term) / (C - B)
    df_dalpha_NT = ((A - B) * ln_term) / (C - B)

    # Total uncertainty
    sigma_f = np.sqrt((df_dalpha * unc) ** 2 + (df_dalpha_NT * sigma_alpha_NT) ** 2)
    
    return sigma_f 

thermalfractions=[]
thermalfractionserrs=[]

for row in table:
    if math.isnan(row['alpha1']) == True:
        thermalfractions.append(x)
        thermalfractionserrs.append(x)
    else:
        thermalfractions.append(tfrac(row['alpha1']))
        thermalfractionserrs.append(tfrac_unc(row['alpha1'],row['b1']))

# results2=Column(tfrac_unc(table['alpha1'],table['b1']),name='error')
results1=Column(thermalfractions,name='data')
results2=Column(thermalfractionserrs,name='error')
tbl=Table([table['source ID'],results1,results2])
print(tbl[:10])

# tbl.write('/users/adignan/tfrac_july.csv',overwrite=True)
# print(np.median(results1))
# print(np.std(results1))
# print(median_absolute_deviation(results1))
# print(median_absolute_deviation(results2))

  source ID           data               error        
------------- ------------------- --------------------
     NGC0337a  0.3026161165873144   0.3967941638508259
     NGC0337b  0.1975276275302447   0.6563770375383026
     NGC0337c 0.41888283127718073 0.053274200047035376
     NGC0337d  0.5674028473328403  0.02601826306095671
      NGC0628                 nan                  nan
NGC0628Enuc.1  0.3664876333899007  0.22321723706624913
NGC0628Enuc.2   0.554464653249289   0.2181909242982021
NGC0628Enuc.3  0.5352483369883572   0.0607496150426901
NGC0628Enuc.4  0.4384973772180985   0.3666001353577992
      NGC0925  0.7513182178914195   1.0500501155547046


/tmp/ipykernel_387282/1224420362.py:45: UserWarning: Warning: converting a masked element to nan.
  if math.isnan(row['alpha1']) == True:
